In [119]:
%%writefile ../../lendingclub/app/app.py
import os
import pandas as pd
from sqlalchemy import create_engine
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output


from lendingclub import config

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


# load api_loans df and historical scores df
disk_engine = create_engine(f'sqlite:///{config.lc_api_db}')
df = pd.read_sql('lc_api_loans',
                 con=disk_engine,
                 parse_dates=[
                     'accept_d', 'exp_d', 'list_d', 'credit_pull_d',
                     'review_status_d', 'ils_exp_d', 'last_seen_list_d'
                 ])
hist_df = pd.read_feather(
    os.path.join(config.data_dir, 'all_eval_loan_info_scored.fth'))

# setup some constants for use in app
rounds = sorted(df['list_d'].dt.hour.unique())
min_date = df['list_d'].min().date()
max_date = df['list_d'].max().date()
print(min_date, max_date)

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H3('API loans'),
    html.Div([
        dcc.DatePickerRange(id='api-loans-date-picker-range',
                            start_date=max_date,
                            end_date=max_date,
                            min_date_allowed=min_date,
                            # for some reason I couldn't select max date
                            max_date_allowed=max_date + pd.DateOffset(days=1),),
        html.Div(id='api-loans-date-picker-range-info')
    ]),
    html.Div([
        dcc.Dropdown(id='api-loans-round-dropdown',
                 options=[{'label':i, 'value': i} for i in rounds],
                 multi=True,
                 value=rounds),
        html.Div(id='api-loans-round-dropdown-info')
    ]),    
    dcc.Graph(id='api-loans-graph', ),
    dash_table.DataTable(
        id='api-table',
        columns=[{
            "name": i,
            "id": i
        } for i in df.columns],
        fixed_columns={
            'headers': True,
            'data': 1
        },
        fixed_rows={
            'headers': True,
            'data': 0
        },
        style_table={
            'maxHeight': '250px',
            'maxWidth': '900px',
            'overflowY': 'scroll',
            'overflowX': 'scroll'
        },
        style_cell={'width': '150px'},
        style_data_conditional=[{
            'if': {
                'row_index': 'odd'
            },
            'backgroundColor': 'rgb(248, 248, 248)'
        }],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold',
            'border': '1px solid pink',
        },
        style_data={},
    ),
    html.H3('CSV loans'),

])

##### CALLBACKS #####
@app.callback(
    Output('api-table', 'data'), 
    [Input('api-loans-date-picker-range', 'start_date'),
     Input('api-loans-date-picker-range', 'end_date'),
     Input('api-loans-round-dropdown', 'value'),]
)
def update_api_table(start_date, end_date, listing_sess):
    sub_df = df.query('list_d >= @start_date and list_d <= @end_date and list_d_hour in @listing_sess')
    return sub_df.to_dict('records')

@app.callback(
    Output('api-loans-date-picker-range-info', 'children'),
    [Input('api-loans-date-picker-range', 'start_date'),
     Input('api-loans-date-picker-range', 'end_date'),]
)
def update_api_loans_date_picker_range_info(start_date, end_date):
    return f'Dates selected from {start_date} to {end_date}'

@app.callback(
    Output('api-loans-round-dropdown-info', 'children'),
    [Input('api-loans-round-dropdown', 'value'),]
)
def update_api_loans_round_dropdown_info(value):
    '''
    So far, haven't been able to find how to sort multi-select dropdown.
    '''
    return f'Selected daily loan release rounds {", ".join([str(i) for i in sorted(value)])}'

@app.callback(
    Output('api-loans-graph', 'figure'), 
    [Input('api-loans-date-picker-range', 'start_date'),
     Input('api-loans-date-picker-range', 'end_date'),
     Input('api-loans-round-dropdown', 'value'),]
)
def update_api_loans_graph(start_date, end_date, listing_sess):
    sub_df = df.query('list_d >= @start_date and list_d <= @end_date and list_d_hour in @listing_sess')
    fig = px.histogram(sub_df, x='catboost_comb_20', nbins=100, histnorm='probability density', color='sub_grade', marginal='rug')
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting ../../lendingclub/app/app.py


In [115]:
import datetime

In [108]:
max_date = df['list_d'].max().date()
max_date

datetime.date(2020, 1, 1)

In [117]:
datetime.date(2020,1,31)+ pd.DateOffset(days=1)

Timestamp('2020-02-01 00:00:00')

In [112]:
max_date + pd.DateOffset(days=1)

Timestamp('2020-01-02 00:00:00')

In [105]:
dcc.DatePickerRange??

In [61]:
rounds = sorted(df['list_d'].dt.hour.unique())

In [98]:
start_date = '2019-12-29'
end_date = '2020-12-30'
sub_df = df.query('list_d >= @start_date and list_d <= @end_date and ')

In [99]:
sub_df

,id,member_id,loan_amount,funded_amount,term,int_rate,exp_default_rate,service_fee_rate,installment,grade,...,list_d_year,list_d_month,list_d_day,list_d_week,list_d_hour,last_seen_list_d_year,last_seen_list_d_month,last_seen_list_d_day,last_seen_list_d_week,last_seen_list_d_hour
0,162593951,204027031,8800.0,7225.0,36,18.620001,12.86,1.52,320.89,D,...,2019,12,29,52,14,2020,1,1,1,1
1,162641707,204068568,32000.0,10675.0,36,8.810000,2.63,0.88,1014.77,A,...,2019,12,29,52,14,2020,1,1,1,14
2,162647001,204073762,16000.0,9800.0,36,20.549999,12.86,1.52,599.12,D,...,2019,12,30,1,18,2020,1,1,1,14
3,162741008,204188195,10000.0,9225.0,36,15.240000,7.96,1.23,347.83,C,...,2019,12,30,1,10,2020,1,1,1,1
6,162889971,204475871,20000.0,0.0,36,8.810000,2.63,0.88,634.23,A,...,2019,12,30,1,18,2020,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,164226584,207083973,35000.0,4925.0,36,14.300000,7.96,1.23,1201.33,C,...,2020,1,1,1,10,2020,1,1,1,14
132,164232075,207089506,15000.0,2650.0,36,12.400000,5.36,1.05,501.09,B,...,2020,1,1,1,14,2020,1,1,1,14
133,164233705,207092146,8000.0,4550.0,36,25.650000,12.86,1.52,320.84,D,...,2020,1,1,1,14,2020,1,1,1,14
134,164235291,207083742,4475.0,0.0,36,25.650000,12.86,1.52,179.47,D,...,2020,1,1,1,14,2020,1,1,1,13


In [66]:
rounds

[6, 10, 14, 18]

In [72]:
dcc.Graph??

In [56]:
df['list_d'].max().date()

datetime.date(2020, 1, 1)

In [31]:
pd.read_sql??